# An Analysis of Childcare Prices Across the United States
## Drake Capstone Project
### Author: Cera Drake
### June 2025

##### Import and load the dataset

In [14]:
import pandas as pd

df = pd.read_csv("nationaldatabaseofchildcareprices.csv", encoding='latin1')  
print(df.head())

  State_Name State_Abbreviation     County_Name  County_FIPS_Code  StudyYear  \
0    Alabama                 AL  Autauga County              1001       2008   
1    Alabama                 AL  Autauga County              1001       2009   
2    Alabama                 AL  Autauga County              1001       2010   
3    Alabama                 AL  Autauga County              1001       2011   
4    Alabama                 AL  Autauga County              1001       2012   

   UNR_16  FUNR_16  MUNR_16  UNR_20to64  FUNR_20to64  ...  MFCCToddler  \
0    5.42     4.41     6.32         4.6          3.5  ...      $83.45    
1    5.93     5.72     6.11         4.8          4.6  ...      $87.39    
2    6.21     5.57     6.78         5.1          4.6  ...      $91.33    
3    7.55     8.13     7.03         6.2          6.3  ...      $95.28    
4    8.60     8.88     8.29         6.7          6.4  ...      $99.22    

   MFCCToddler_flag  MFCCPreschool  MFCCPreschool_flag  _75FCCInfant  \
0 